In [1]:
import pandas as pd; pd.set_option('display.max_colwidth',3000)
#import matplotlib.pyplot as plt
#import numpy as np
#from sklearn.decomposition import FactorAnalysis
#from sklearn.preprocessing import StandardScaler

In [9]:
#open file
df = pd.read_excel(r"C:\Users\mcove\Downloads\fake_data.xlsx")

#strip whitespace
for x in df.iloc[:,3:].columns:
    df.loc[:,x] = df[x].str.strip()

#recode
df.iloc[:,3:] = df.iloc[:,3:].replace(['Strongly Agree','Agree','Neither Agree nor Disagree','Disagree','Strongly Disagree'],[5,4,3,2,1])

#recode
unfav = df.copy()
unfav.iloc[:,3:] = unfav.iloc[:,3:].replace([5,4,3,2,1],['Agree','Agree',2,'Disagree','Disagree'])

#items
items = df.iloc[:,3:]

In [20]:
#testing
cols = list(df.iloc[:,3:].columns)[:-1]
df_t = pd.DataFrame()
for col in cols:
    df_temp = group_mean_differences(dataframe=df,grouping_variables_list=['C / T'],DV_col=df[col].name,equal_var=True,subset_combinations=0)
    df_t = pd.concat([df_t,df_temp])
df_t.to_excel(r"C:\Users\mcove\Downloads\df_t_subset_combo0.xlsx")
df_t

,grouping_variable,group,subset,DV,N,n,t,p,group_mean,mean_difference,median_difference
0,C / T,Treatment,[],Meetings I run achieve the meeting objectives,191,303,-0.506459,0.61312,4.178571,-0.036618,0.0
1,C / T,Control,[],Meetings I run achieve the meeting objectives,191,270,0.506459,0.61312,4.215190,0.036618,0.0
0,C / T,Treatment,[],Meetings I run begin and end on time,191,303,0.248948,0.80367,3.982143,0.032776,0.0
1,C / T,Control,[],Meetings I run begin and end on time,191,270,-0.248948,0.80367,3.949367,-0.032776,0.0
0,C / T,Treatment,[],Meetings I run end with clear next steps,191,303,0.217836,0.82779,4.008929,0.021587,0.0
1,C / T,Control,[],Meetings I run end with clear next steps,191,270,-0.217836,0.82779,3.987342,-0.021587,0.0
0,C / T,Treatment,[],Meetings that others run achieve the meeting objectives,191,303,1.500994,0.13503,3.375000,0.147152,0.0
1,C / T,Control,[],Meetings that others run achieve the meeting objectives,191,270,-1.500994,0.13503,3.227848,-0.147152,0.0
0,C / T,Treatment,[],Meetings that others run begin and end on time,191,303,0.990891,0.32301,3.223214,0.121948,0.0
1,C / T,Control,[],Meetings that others run begin and end on time,191,270,-0.990891,0.32301,3.101266,-0.121948,0.0


In [33]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
group_mean_differences = None
def group_mean_differences(dataframe,DV_col,grouping_variables_list,equal_var=True,subseter_variables_list=[],subset_combinations=0):
    import scipy.stats as stats
    import itertools
    def get_key_from_value_list(my_dict,value):
        for k, v_list in my_dict.items():
            for v in v_list:
                if v == value:
                    return k
    def get_keys_list_from_value_list(my_dict,search_list):
        k_list = []
        for k, v_list in my_dict.items():
            for s in search_list:
                if s in v_list:
                    if k not in k_list:
                        k_list.append(k)
        return k_list
    data = dataframe.copy()
    
    #Create dictionary d of {subseter variable:[categories]} and list of categories
    subset_dict = {}
    for sv in subseter_variables_list:
        subset_dict.update({sv:list(data[sv].unique())})
    categories = [v for value_lists in subset_dict.values() for v in value_lists]
    print('subset_dict = ', subset_dict)
    print('categories = ', categories)

    #create subset_list
    subset_list = []
    for i in range(subset_combinations+1):
        print(i)
        for subset in itertools.combinations(categories, i):
            groupings = [get_key_from_value_list(subset_dict,value) for value in subset]
            #print(subset)
            category_counts = [groupings.count(x) for x in groupings]
            print('category_counts = ',category_counts)
            repeated_categories = any(x > 1 for x in category_counts)
            print(not repeated_categories)
            #print(subset not in subset_list)
            if not repeated_categories and subset not in subset_list:
                subset_list.append(list(subset))
    print('subset_list = ',subset_list)

    #Add dummy coded variables for subseter variables
#    for gv in subseter_variables_list:
#        groups = data[gv].unique()
#        for g in groups:
#            data[g] = np.where(data[gv] == g, 1, 0)
            
    #Add dummy coded variables for grouping variables
#    for gv in grouping_variables_list:
#        groups = data[gv].unique()
#        for g in groups:
#            data[g] = np.where(data[gv] == g, 1, 0)
    
    #Create DataFrame containing statistics
    df_returned = pd.DataFrame()#columns=['grouping_variable','group','subset','DV','N','n','t','p'])
    for gv in grouping_variables_list:
        groups = data[gv].unique()
        for g in groups:
            #get statistics for subsets of dataset
            for subset in subset_list:
                #print(subset)
                subseter_variables = get_keys_list_from_value_list(subset_dict,subset)
                b_subset = data[subseter_variables].isin(subset)#.all(axis='columns') This threw an errors when trying to use it a second time after pd.concat below
                g1b = data[gv] == g
                g2b = data[gv] != g
                
                group1 = data.loc[g1b&b_subset.all(axis='columns'),DV_col]
                group2 = data.loc[g2b&b_subset.all(axis='columns'),DV_col]
                N = group1.count()+group2.count()
                n = pd.concat([b_subset,g1b]).all(axis='columns').sum()
                temp_df = pd.DataFrame({'grouping_variable':[gv],
                                        'group':[g],
                                        'subset':[str(subset)],
                                        'DV':[DV_col],
                                        'N':[N],
                                        'n':[n]
                                        })
                if group1.notna().any() and group2.notna().any():
                    group_mean = group1.mean()
                    mean_difference = group_mean - group2.mean()
                    median_difference = group1.median() - group2.median()
                    ttest = stats.ttest_ind(a=group1, b=group2, equal_var=equal_var)
                    t = ttest[0]
                    p = ttest[1]
                    temp_df['t'] = t
                    temp_df['p'] = p.round(5)
                    temp_df['group_mean'] = group_mean
                    temp_df['mean_difference'] = mean_difference
                    temp_df['median_difference'] = median_difference
                    
                #else:
                   # print('missing values')
                   # print('\t',g,subset)
                df_returned = pd.concat([df_returned,temp_df])

    #return df_returned.sort_values('p').reset_index(drop=True)

In [77]:
group_mean_differences(dataframe=df,grouping_variables_list=['C / T'],DV_col=df[col].name,equal_var=True,subseter_variables_list=['Country'],subset_combinations=3)

subset_dict =  {'Country': ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']}
categories =  ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']
0
category_counts =  []
True
1
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
category_counts =  [1]
True
2
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_counts =  [2, 2]
False
category_co

In [55]:
#testing
cols = list(df.iloc[:,3:].columns)[:-1]
df_t = pd.DataFrame()
for col in cols:
    df_temp = group_mean_differences(dataframe=df,grouping_variables_list=['C / T'],DV_col=df[col].name,equal_var=True,subseter_variables_list=['Country'],subset_combinations=3)
    df_t = pd.concat([df_t,df_temp])
#df_t.to_excel(r"C:\Users\mcove\Downloads\df_t_subset_combo2.xlsx")
df_t

subset_dict {'Country': ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']}
categories ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']
subset_dict {'Country': ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']}
categories ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']
subset_dict {'Country': ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']}
categories ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']
subset_dict {'Country': ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']}
categories ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']
subset_dict {'Country': ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 'IN', 'IL', 'SG', 'CR', 'NG']}
categories ['MX', 'GB', 'US', 'HK', 'PH', 'UY', 'HU', 'CN', 'BD', 

,grouping_variable,group,subset,DV,N,n,t,p,group_mean,mean_difference,median_difference
0,C / T,Treatment,['GB'],Meetings I run achieve the meeting objectives,21,133,-0.795822,0.43597,4.000000,-0.142857,0.0
1,C / T,Control,['GB'],Meetings I run achieve the meeting objectives,21,100,0.795822,0.43597,4.142857,0.142857,0.0
2,C / T,Treatment,['HK'],Meetings I run achieve the meeting objectives,7,119,-0.845154,0.43659,3.750000,-0.250000,0.0
3,C / T,Treatment,['IN'],Meetings I run achieve the meeting objectives,7,119,-0.845154,0.43659,3.750000,-0.250000,0.0
4,C / T,Control,['HK'],Meetings I run achieve the meeting objectives,7,86,0.845154,0.43659,4.000000,0.250000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25,C / T,Control,['BD'],I am satisfied with my involvement in decisions that affect my work.,2,81,NaN,NaN,1.000000,-3.000000,-3.0
26,C / T,Control,['IL'],I am satisfied with my involvement in decisions that affect my work.,2,81,NaN,NaN,2.000000,-2.000000,-2.0
27,C / T,Control,['SG'],I am satisfied with my involvement in decisions that affect my work.,2,81,NaN,NaN,4.000000,1.000000,1.0
28,C / T,Control,['CR'],I am satisfied with my involvement in decisions that affect my work.,3,82,NaN,NaN,3.000000,0.000000,0.0
